<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Заполнение-пропусков" data-toc-modified-id="Заполнение-пропусков-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Заполнение пропусков</a></span></li><li><span><a href="#Прямое-кодирование" data-toc-modified-id="Прямое-кодирование-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Прямое кодирование</a></span></li><li><span><a href="#Масштабирование-признаков" data-toc-modified-id="Масштабирование-признаков-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Масштабирование признаков</a></span></li></ul></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

## Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

Github: https://github.com/maria-okoledova394/supervised-learning.git

## Подготовка данных

In [ ]:
pip install pandas-profiling

In [ ]:
pip install tqdm

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import pandas_profiling
import matplotlib.pyplot as plt
from sklearn import tree
from tqdm.notebook import trange, tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score

In [ ]:
df = pd.read_csv('./datasets/Churn.csv')

pandas_profiling.ProfileReport(df)

### Заполнение пропусков
Пропуски в столбце `Tenure` (сколько лет человек является клиентом банка) составляют 9.1%. Это слишком много, чтобы удалить строки с пропущенными значениями. Заменим пропуски медианой.

In [ ]:
df.loc[df['Tenure'].isna(), 'Tenure'] = df['Tenure'].median()

df['Tenure'].isna().sum()

Подготовим признаки:

- Не все признаки нужны для обучения модели. Исключим `RowNumber`, `CustomerId`, `Surname`;
- Преобразуем категориальные признаки `Geography`, `Gender` в численные;
- Масштабируем численные признаки `CreditScore`, `Age`, `Tenure`, `Balance`, `NumOfProducts`, `EstimatedSalary`.

In [ ]:
# исключаем лишние столбцы 

df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

### Прямое кодирование

Используем технику прямого кодирование для преобразования категориальных столбцов.

In [ ]:
df_ohe = pd.get_dummies(df, drop_first=True)

df_ohe.columns

### Масштабирование признаков

Используем метод масштабирование - стандартизация данных.

In [ ]:
# разделим сначала на обучающую и валидационную + тестовую
df_train, df_valid_test = train_test_split(df_ohe, test_size=0.4, stratify=df['Exited'], random_state=12345)

# разделим валидационную + тестовую на валидационную и тестовую
df_valid, df_test = train_test_split(df_valid_test, test_size=0.5, stratify=df_valid_test['Exited'], random_state=12345)

# признаки обучающей выборки
features_train = df_train.drop('Exited', axis=1)
target_train = df_train['Exited']

# признаки валидационной выборки
features_valid = df_valid.drop('Exited', axis=1)
target_valid = df_valid['Exited']

# признаки тестовой выборки
features_test = df_test.drop('Exited', axis=1)
target_test = df_test['Exited']

# масштабирование признаков
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

scaler = StandardScaler()
scaler.fit(features_train[numeric])

features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

Закончили подготовку данных.

## Исследование задачи

In [ ]:
features_zeros = features_train[target_train == 0]['Age'].count()
features_ones = features_train[target_train == 1]['Age'].count()

print(features_zeros/features_ones)

В нашей задаче дисбаланс классов. В данных 10000 наблюдений, из которых 7963 (79.6%) относятся к нулевому классу и 2037 (20.4%) – к единичному. Соотношение положительного и отрицательного классов примерно 1:4. Это плохо влияет на обучение модели. 

Обучим сначала модели дерева решений, случайного леса и логистической регрессии без учета дисбаланса классов.

In [ ]:
def get_model_metrics(model, features, target, features_test, target_test):
    model.fit(features, target)
    predicted = model.predict(features_test)
    
    recall = recall_score(target_test, predicted)
    precision = precision_score(target_test, predicted)
    f1 = f1_score(target_test, predicted)
    
    probabilities = model.predict_proba(features_test)
    probabilities_one = probabilities[:, 1]
    auc_roc = roc_auc_score(target_test, probabilities_one)
    
    return recall, precision, f1, auc_roc

def create_df_mertics(tree = [], forest = [], log_reg = []):
    columns = ['model', 'recall', 'precision', 'f1', 'auc_roc']
    metrics = [tree, forest, log_reg]

    return pd.DataFrame(data=metrics, columns=columns)

In [ ]:
# дерево решений
model_tree = DecisionTreeClassifier(random_state=12345)
recall_tree, precision_tree, f1_tree, auc_roc_tree = get_model_metrics(
    model_tree, features_train, target_train, features_valid, target_valid)
tree = ['model_tree', recall_tree, precision_tree, f1_tree, auc_roc_tree]

# случайный лес
model_forest = RandomForestClassifier(random_state=12345)
recall_forest, precision_forest, f1_forest, auc_roc_forest = get_model_metrics(
    model_forest, features_train, target_train, features_valid, target_valid)
forest = ['model_forest', recall_forest, precision_forest, f1_forest, auc_roc_forest]

# логистическая регрессия
model_log_reg = LogisticRegression(random_state=12345)
recall_log_reg, precision_log_reg, f1_log_reg, auc_roc_log_reg = get_model_metrics(
    model_log_reg, features_train, target_train, features_valid, target_valid)
log_reg = ['model_log_reg', recall_log_reg, precision_log_reg, f1_log_reg, auc_roc_log_reg]

display(create_df_mertics(tree, forest, log_reg))

Лучшие результаты F1-меры = 0.61, AUC-ROC = 0.86 у модели случайного леса.

## Борьба с дисбалансом

Избавимся от дисбаланса классов, чтобы получить более высокое значение F1-меры. Добавим параметр class_weight='balanced' моделям, чтобы они учитывали вес классов.

In [ ]:
# дерево решений
model_tree_balanced = DecisionTreeClassifier(random_state=12345, class_weight='balanced')
recall_tree_balanced, precision_tree_balanced, f1_tree_balanced, auc_roc_tree_balanced = get_model_metrics(
    model_tree_balanced, features_train, target_train, features_valid, target_valid)
tree_balanced = ['model_tree_balanced', recall_tree_balanced, precision_tree_balanced, f1_tree_balanced, auc_roc_tree_balanced]

# случайный лес
model_forest_balanced = RandomForestClassifier(random_state=12345, class_weight='balanced')
recall_forest_balanced, precision_forest_balanced, f1_forest_balanced, auc_roc_forest_balanced = get_model_metrics(
    model_forest_balanced, features_train, target_train, features_valid, target_valid)
forest_balanced = ['model_forest_balanced', recall_forest_balanced, precision_forest_balanced, f1_forest_balanced, 
                   auc_roc_forest_balanced]

# логистическая регрессия
model_log_reg_balanced = LogisticRegression(random_state=12345, class_weight='balanced')
recall_log_reg_balanced, precision_log_reg_balanced, f1_log_reg_balanced, auc_roc_log_reg_balanced = get_model_metrics(
    model_log_reg_balanced, features_train, target_train, features_valid, target_valid)
log_reg_balanced = ['model_log_reg_balanced', recall_log_reg_balanced, precision_log_reg_balanced, f1_log_reg_balanced, 
                    auc_roc_log_reg_balanced]

display(create_df_mertics(tree_balanced, forest_balanced, log_reg_balanced))

F1-мера улучшилась у случайного дерева и логистической регрессии, но уменьшилась у случайного дерева. Попробуем увеличить выборку - сделать объекты редкого класса не такими редкими в данных.

In [ ]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [ ]:
features_train_upsampled, target_train_upsampled = upsample(features_train, target_train, 4)

# дерево решений
model_tree_upsampled = DecisionTreeClassifier(random_state=12345)
recall_tree_upsampled, precision_tree_upsampled, f1_tree_upsampled, auc_roc_tree_upsampled = get_model_metrics(
    model_tree_upsampled, features_train_upsampled, target_train_upsampled, features_valid, target_valid)
tree_upsampled = ['model_tree_downsampled', recall_tree_upsampled, precision_tree_upsampled, f1_tree_upsampled, 
                  auc_roc_tree_upsampled]

# случайный лес
model_forest_upsampled = RandomForestClassifier(random_state=12345)
recall_forest_upsampled, precision_forest_upsampled, f1_forest_upsampled, auc_roc_forest_upsampled = get_model_metrics(
    model_forest_upsampled, features_train_upsampled, target_train_upsampled, features_valid, target_valid)
forest_upsampled = ['model_forest_downsampled', recall_forest_upsampled, precision_forest_upsampled, f1_forest_upsampled, 
                    auc_roc_forest_upsampled]

# логистическая регрессия
model_log_reg_upsampled = LogisticRegression(random_state=12345)
recall_log_reg_upsampled, precision_log_reg_upsampled, f1_log_reg_upsampled, auc_roc_log_reg_upsampled = get_model_metrics(
    model_log_reg_upsampled, features_train_upsampled, target_train_upsampled, features_valid, target_valid)
log_reg_upsampled = ['model_log_reg_downsampled', recall_log_reg_upsampled, precision_log_reg_upsampled, f1_log_reg_upsampled, 
                     auc_roc_log_reg_upsampled]

display(create_df_mertics(tree_upsampled, forest_upsampled, log_reg_upsampled))
target_train_upsampled.value_counts()

Стало лучше. Попробуем другую технику, уменьшить выборку - сделать объекты частого класса не такими частыми.

In [ ]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

In [ ]:
features_train_downsampled, target_train_downsampled = downsample(features_train, target_train, 0.25)

# дерево решений
model_tree_downsampled = DecisionTreeClassifier(random_state=12345)
recall_tree_downsampled, precision_tree_downsampled, f1_tree_downsampled, auc_roc_tree_downsampled = get_model_metrics(
    model_tree_downsampled, features_train_downsampled, target_train_downsampled, features_valid, target_valid)
tree_downsampled = ['model_tree_downsampled', recall_tree_downsampled, precision_tree_downsampled, f1_tree_downsampled, auc_roc_tree_downsampled]

# случайный лес
model_forest_downsampled = RandomForestClassifier(random_state=12345)
recall_forest_downsampled, precision_forest_downsampled, f1_forest_downsampled, auc_roc_forest_downsampled = get_model_metrics(
    model_forest_downsampled, features_train_downsampled, target_train_downsampled, features_valid, target_valid)
forest_downsampled = ['model_forest_downsampled', recall_forest_downsampled, precision_forest_downsampled, f1_forest_downsampled, auc_roc_forest_downsampled]

# логистическая регрессия
model_log_reg_downsampled = LogisticRegression(random_state=12345)
recall_log_reg_downsampled, precision_log_reg_downsampled, f1_log_reg_downsampled, auc_roc_log_reg_downsampled = get_model_metrics(
    model_log_reg_downsampled, features_train_downsampled, target_train_downsampled, features_valid, target_valid)
log_reg_downsampled = ['model_log_reg_downsampled', recall_log_reg_downsampled, precision_log_reg_downsampled, f1_log_reg_downsampled, auc_roc_log_reg_downsampled]

display(create_df_mertics(tree_downsampled, forest_downsampled, log_reg_downsampled))
target_train_downsampled.value_counts()

Лучшие результаты у модели случайного леса после увеличения выборки. Подберем лучшую модель, меняя параметры.

In [ ]:
best_model_forest_upsampled = None
best_f1_forest_upsampled = 0
best_est_forest_upsampled = 0
best_depth_forest_upsampled = 0
best_auc_roc_forest_upsampled = 0
best_recall_forest_upsampled = 0

for est in trange(1, 100, 10):
    for depth in range (1, 20):
        model_forest_upsampled = RandomForestClassifier(n_estimators=est, max_depth=depth, random_state=12345)
        
        recall_forest_upsampled, precision_forest_upsampled, f1_forest_upsampled, auc_roc_forest_upsampled = get_model_metrics(
            model_forest_upsampled, features_train_upsampled, target_train_upsampled, features_valid, target_valid)
        
        if f1_forest_upsampled > best_f1_forest_upsampled:
            best_model_forest_upsampled = model_forest_upsampled
            best_f1_forest_upsampled = f1_forest_upsampled
            best_est_forest_upsampled = est
            best_depth_forest_upsampled = depth
            best_auc_roc_forest_upsampled = auc_roc_forest_upsampled
            best_recall_forest_upsampled = recall_forest_upsampled
            best_precision_forest_upsampled = precision_forest_upsampled
            
print("F1 наилучшей модели дерева на валидационной выборке:", best_f1_forest_upsampled, "Количество деревьев:", best_est_forest_upsampled, "Максимальная глубина:", best_depth_forest_upsampled)
display(create_df_mertics(['best_model_forest_upsampled', best_recall_forest_upsampled, best_precision_forest_upsampled, best_f1_forest_upsampled, best_auc_roc_forest_upsampled]).loc[[0]])

Вывод:

Больше всего f1-мера у модели случайного леса с параметрами:
- `n_estimators` = 91 (количество деревьев);
- `max_depth` = 11 (максимальная глубина дерева).

## Тестирование модели

Отдельные обучающие и валидационная выборки были нужны для подбора гиперпараметров. Теперь, чтобы улучшить результаты, обучим модель обучающей + валидационной выборке.

In [ ]:
features_train_valid = pd.concat([features_train, features_valid], ignore_index=True)
target_train_valid = pd.concat([target_train, target_valid], ignore_index=True)

features_train_valid_upsampled, target_train_valid_upsampled = upsample(features_train_valid, target_train_valid, 4)

model_forest_test = RandomForestClassifier(n_estimators=best_est_forest_upsampled, max_depth=best_depth_forest_upsampled, 
                                           random_state=12345)
recall_forest_test, precision_forest_test, f1_forest_test, auc_roc_forest_test = get_model_metrics(
    model_forest_test, features_train_valid_upsampled, target_train_valid_upsampled, features_test, target_test)

print("F1 наилучшей модели случайного леса на тестовой выборке:", f1_forest_test)
display(create_df_mertics(['best_model_forest_upsampled', recall_forest_test, precision_forest_test, 
                           f1_forest_test, auc_roc_forest_test]).loc[[0]])

F1-мера на тестовой выборке 0.61. AUC-ROC лучше, чем у случайной модели, но еще далека до 1.